In [1]:
import stanza
import numpy as np
import pandas as pd

In [2]:
notesdf = pd.read_excel('P:/ORD_Singh_201911038D/Justin/InpatNote.xls')
nlp = stanza.Pipeline('en', package='mimic', processors={'ner' : 'i2b2'}, ner_batch_size=512)
print("DataFrame columns: ", ','.join([x for x in notesdf.columns]))

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


2021-06-24 12:45:33 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | mimic   |
| pos       | mimic   |
| lemma     | mimic   |
| depparse  | mimic   |
| ner       | i2b2    |

2021-06-24 12:45:33 INFO: Use device: cpu
2021-06-24 12:45:33 INFO: Loading: tokenize
2021-06-24 12:45:33 INFO: Loading: pos
2021-06-24 12:45:34 INFO: Loading: lemma
2021-06-24 12:45:34 INFO: Loading: depparse
2021-06-24 12:45:35 INFO: Loading: ner
2021-06-24 12:45:38 INFO: Done loading processors!


DataFrame columns:  StudyID,ReportText,TIUStandardTitle,TIUDocumentDefinition,TIUDocumentSID


In [3]:
print(len(['_'.join(x) for x in zip(notesdf.StudyID.astype(str), notesdf.TIUDocumentSID.astype(str))]))
print(len(np.unique(['_'.join(x) for x in zip(notesdf.StudyID.astype(str), notesdf.TIUDocumentSID.astype(str))])))

2730
2730


In [4]:
notesdf.ReportText = notesdf.ReportText.str.replace("\s+", " ", regex=True).str.replace("'", "").str.replace('"', '')

In [5]:
%%time
testdoc = nlp(notesdf.ReportText.iloc[0])
for x in testdoc.entities[:5]:
    print(x.type, '\t\t', x.text)

PROBLEM 		 dizziness
PROBLEM 		 vomiting
PROBLEM 		 Vomitus
PROBLEM 		 associated cold sweats
PROBLEM 		 these symptoms
Wall time: 29.9 s


In [6]:
%%time
entities_by_studydocid = {'_'.join(x):[] for x in zip(notesdf.StudyID.astype(str), notesdf.TIUDocumentSID.astype(str))}

for row in notesdf.itertuples():
    text = row.ReportText
    studydocid = f'{row.StudyID}_{row.TIUDocumentSID}'
    entities = nlp(text).entities
    entities_by_studydocid[studydocid] += [entities]

Wall time: 12h 11min 30s


In [7]:
import pickle
with open("note_entities_by_studydocid.pkl",'wb') as outfile:
    pickle.dump(entities_by_studydocid, outfile)


In [14]:
from sklearn.preprocessing import MinMaxScaler

#Load the old data set for which we have labels
structdf = pd.read_csv("input.csv")
structdf = structdf.drop('smk_flags_man', axis=1)
structdf = structdf.drop('smk_flags_db', axis=1)
structdf = structdf[structdf.mod_label != 'INVALID']
print(f'Columns in df:\n{", ".join([x for x in structdf.columns])}')
label_column = 'mod_label'
feature_columns = list(structdf.columns)
feature_columns.remove(label_column)
feature_columns_cont = [
    'ed_inp_delta'
    ,'ed_duration'
    ,'elix_score'
    ,'age_val'
]
feature_columns_cat = [
    'img_flags'
    ,'con_flags'
]

structdf['ed_inp_delta'] = MinMaxScaler().fit_transform(structdf['ed_inp_delta'].values.reshape(-1,1))
structdf['ed_duration'] = MinMaxScaler().fit_transform(structdf['ed_duration'].values.reshape(-1,1))
structdf['elix_score'] = MinMaxScaler().fit_transform(structdf['elix_score'].values.reshape(-1,1))
structdf['age_val'] = MinMaxScaler().fit_transform(structdf['age_val'].values.reshape(-1,1))

Columns in df:
study_ids, ed_duration, ed_inp_delta, age_val, img_flags, con_flags, elix_score, mod_label


In [16]:
nonotesids = set()
for ids in structdf.study_ids:
    if str(ids) not in [x.split('_')[0] for x in entities_by_studydocid.keys()]:
        nonotesids.add(ids)
print(len(nonotesids))

25


In [18]:
retaineddf = structdf[~structdf.study_ids.isin(nonotesids)]
retaineddf.shape

(192, 8)

In [19]:
def extract_entities_for_studyid(entity_holder, studyid):
    returned_list = []
    studyid = str(studyid)
    for key,values in entity_holder.items():
        if studyid == key.split('_')[0]:
            values = values[0]  #always a list of lists
            for entity in values:
                returned_list.append(entity.text.lower())
    return returned_list

In [21]:
extract_entities_for_studyid(entities_by_studydocid, 2003)[:10]

['chest and sinus congestion',
 'weakness',
 'dizziness',
 'cp',
 'sob',
 'blood pressure',
 'pulse',
 'respirations',
 'temperature',
 'spo2']

In [75]:
studyidents = {x:[] for x in retaineddf.study_ids}
entstrings = []
for key in studyidents:
    studyidents[key] += extract_entities_for_studyid(entities_by_studydocid, key)
    #if studyidents[key] == []:
    #    continue #skip adding to entstrings, we'll remove the key from studyidents shortly
    entstrings += [' '.join(studyidents[key])]
print(entstrings[0]) #should basically be a "raw" corpus where the corpus is made up of extracted text using StanzaNLP clinical model; CountVectorizer expects this format

chest and sinus congestion weakness dizziness cp sob blood pressure pulse respirations temperature spo2 pulse oximetry pain scale medications fall precaution measures falls fall precautions wt temp pulse resp pulse oximetry pain cough dizzy weakness cough mild sob some nasal congestion generally weak chest pain tightness falls focal weakness n v d stools and urine melena hemoptysis dizzy falls cough weakness motrin knee pain naproxen hypothyroidism snomed ct hearing loss icd unspecified gingival and periodontal disease icd hyperlipidemia snomed ct vitamin b deficiency osteoarthritis of knee snomed ct colonic polyps primary malignant neoplasm of prostate snomed ct paresthesias icd pain in joint involving shoulder region active outpatient medications cyanocobalamin inj inject 1000mcg active (1ml) anemia levothyroxine na (synthroid) thyroid replacement simvastatin naproxen pain simvastatin cholesterol 3) syringe tamsulosin hcl urination terbinafine hcl fungal infection non-va aspirin motr

In [98]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score

In [77]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(entstrings)
print(' | '.join(np.asarray(vectorizer.get_feature_names())[np.argsort(np.sum(X.toarray(), axis=0))[-100:]][::-1]))
print(X.toarray().shape)

pain | active | blood | icd | the | medications | pressure | of | ct | take | dizziness | tab | pulse | non | in | headache | cap | hcl | symptoms | va | chest | bp | acute | any | one | heart | dl | snomed | and | his | medication | chronic | your | weakness | diabetes | ekg | nausea | weight | changes | or | left | disease | outpatient | head | calcium | vomiting | fever | right | lisinopril | hypertension | glucose | sct | mouth | temp | signs | na | edema | iv | stroke | aspirin | vision | temperature | eval | disorder | breath | to | loss | nasal | other | capsule | score | issu | acetaminophen | cholesterol | mg | cough | urine | oximetry | total | oral | insulin | distress | infection | evaluation | labs | tablet | dizzy | abdominal | rate | sob | mild | vital | vitamin | atorvastatin | htn | shortness | assessment | skin | treatment | ear
(192, 5226)


In [78]:
stopwords = ['the', 'of', 'in', 'and', 'your', 'or', 'to'] #manually added just looking at the top 100 most common words

In [82]:
vectorizer = CountVectorizer(stop_words=stopwords)
X = vectorizer.fit_transform(entstrings)
print(' | '.join(np.asarray(vectorizer.get_feature_names())[np.argsort(np.sum(X.toarray(), axis=0))[-100:]][::-1]))
print(X.toarray().shape)
print(structdf.shape)

pain | active | blood | icd | medications | pressure | ct | take | dizziness | tab | pulse | non | headache | cap | hcl | symptoms | va | chest | bp | acute | any | one | dl | heart | snomed | his | medication | chronic | weakness | diabetes | nausea | ekg | changes | weight | disease | left | outpatient | head | calcium | fever | vomiting | right | lisinopril | hypertension | glucose | sct | mouth | temp | signs | na | edema | iv | vision | aspirin | stroke | temperature | eval | disorder | breath | nasal | loss | other | capsule | acetaminophen | score | issu | cholesterol | mg | cough | urine | oximetry | total | oral | insulin | distress | evaluation | infection | dizzy | labs | tablet | sob | rate | abdominal | mild | htn | shortness | vital | atorvastatin | vitamin | skin | assessment | ear | treatment | neck | exam | o2 | level | ha | by | chills
(192, 5219)
(217, 8)


In [150]:
test_aucs = []
train_aucs = []
test_reports = []
train_reports = []
for i in range(100):
    eX_train, eX_test, ey_train, ey_test = train_test_split(retaineddf.drop("mod_label", axis=1), \
                                                            retaineddf["mod_label"], \
                                                            test_size=0.8, random_state=42)
    model = LogisticRegression(solver='liblinear', penalty='l2', C=0.55, max_iter=1000, fit_intercept = True)
    etest = eX_test.iloc[:,:]
    etrain = eX_train.iloc[:,:]
    model.fit(etrain, ey_train)
    test_probas  = model.predict_proba(etest)
    test_preds = model.predict(etest)
    train_preds = model.predict(etrain)
    train_probas = model.predict_proba(etrain)
    testauc = roc_auc_score(ey_test, test_probas[:,1])
    trainauc = roc_auc_score(ey_train, train_probas[:,1])
    test_aucs += [testauc]
    train_aucs += [trainauc]
    test_reports += [classification_report(ey_test, test_preds, output_dict=True)]
    train_reports += [classification_report(ey_train, train_preds, output_dict=True)]

print(f" Average Test AUC \u00B1 \u03C3 (100 iterations): " +\
      f"{np.average(test_aucs):0.3f} \u00B1 {np.std(test_aucs):0.3f}")
print(f"Average Train AUC \u00B1 \u03C3 (100 iterations): " +\
      f"{np.average(train_aucs):0.3f} \u00B1 {np.std(train_aucs):0.3f}")
'''
skf = StratifiedKFold(n_splits=5, shuffle=False)
for train_idx, test_idx in skf.split(retaineddf.drop("mod_label", axis=1), retaineddf.mod_label):
    X_train = retaineddf.drop("mod_label", axis=1).values[train_idx]
    y_train = retaineddf.mod_label.iloc[train_idx]
    MinMaxScaler().fit_transform(structdf['ed_inp_delta'].values.reshape(-1,1)
    clf = LogisticRegression()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(retaineddf.drop("mod_label", axis=1).values[test_idx])
    print(classification_report(retaineddf.mod_label.iloc[test_idx], y_pred))
    print()
''';

 Average Test AUC ± σ (100 iterations): 0.624 ± 0.000
Average Train AUC ± σ (100 iterations): 0.795 ± 0.000


In [149]:
test_aucs = []
train_aucs = []
test_reports = []
train_reports = []
for i in range(100):
    eX_train, eX_test, ey_train, ey_test = train_test_split(pd.concat([retaineddf.reset_index().drop("mod_label", axis=1), pd.DataFrame(X.toarray())], axis=1), \
                                                            retaineddf["mod_label"], \
                                                            test_size=0.8, random_state=42)
    model = LogisticRegression(solver='liblinear', penalty='l1', C=10, max_iter=1000, fit_intercept = False)
    etest = eX_test.iloc[:,:]
    etrain = eX_train.iloc[:,:]
    model.fit(etrain, ey_train)
    test_probas  = model.predict_proba(etest)
    test_preds = model.predict(etest)
    train_preds = model.predict(etrain)
    train_probas = model.predict_proba(etrain)
    testauc = roc_auc_score(ey_test, test_probas[:,1])
    trainauc = roc_auc_score(ey_train, train_probas[:,1])
    test_aucs += [testauc]
    train_aucs += [trainauc]
    test_reports += [classification_report(ey_test, test_preds, output_dict=True)]
    train_reports += [classification_report(ey_train, train_preds, output_dict=True)]

print(f" Average Test AUC \u00B1 \u03C3 (100 iterations): " +\
      f"{np.average(test_aucs):0.3f} \u00B1 {np.std(test_aucs):0.3f}")
print(f"Average Train AUC \u00B1 \u03C3 (100 iterations): " +\
      f"{np.average(train_aucs):0.3f} \u00B1 {np.std(train_aucs):0.3f}")
'''
skf = StratifiedKFold(n_splits=5, shuffle=False)
for train_idx, test_idx in skf.split(retaineddf.drop("mod_label", axis=1), retaineddf.mod_label):
    entcounts_normalized = MinMaxScaler().fit_transform(X.toarray())
    X_train = np.hstack((retaineddf.drop("mod_label", axis=1).values[train_idx], entcounts_normalized[train_idx]))
    y_train = retaineddf.mod_label.iloc[train_idx]
    clf = LogisticRegression(C=1,max_iter=10000)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(np.hstack((retaineddf.drop("mod_label", axis=1).values[test_idx], entcounts_normalized[test_idx])))
    print(classification_report(retaineddf.mod_label.iloc[test_idx], y_pred))
    print()
''';

 Average Test AUC ± σ (100 iterations): 0.515 ± 0.001
Average Train AUC ± σ (100 iterations): 1.000 ± 0.000
